
# Exercises: Clustering

In [2]:
# Import necessary libraries here
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt

## Wine dataset
This exercise is base on the wine dataset included in the sklearn.datasets module.
In the Jupyter tab explore the 'wine' object. It stores all kinds of data about the wine dataset. You will probably need data, feature_names, target and target_names. In the following cell the wine dataset is loaded.

 ### Data preperation

In [15]:
# The dataset is loaded for you.
wine = load_wine()
X = pd.DataFrame(wine.data, columns=wine.feature_names)
y = pd.Series(wine.target)
X.head()
y.head()

a. It is often a good idea to normalize the dataset. Normalization is a statistical method that helps mathematical-based algorithms to interpret features with different magnitudes and distributions equally. One of the most used normalization techniques is transforming your input data to a set with a mean of 0 and a standard deviation of 1. (calculating Z-scores).\
In sklearn this can be done with the StandardScaler.\
Create a new dataframe with the normalized data.


### Model selection and hyperparameter selection
 a.  Fit a K-means with 3 clusters on this set. Use random_state=42 to ensure the same results every time.\



b. Derive the model and predict the clusters.

c. Instead of performing the Scaler seperatly, use the pipeline module from sklearn. A Pipeline is a sequence of data tranformers (in this case the StandardScaler() with an optional final predictor (in this case KMeans). You already used pipeline to transform a polynominal regression to a linear regression.  \
Create a pipeline that first executes the StandardScaler() and uses the output to apply Kmeans and the KMeans.

d. Create a dataframe with the resulting 'clusters' and the original 'wine classes'.\
e. Can you see a pattern when comparing the clusters with the original wine classes?

e. Visualize the clusters by plotting the first two features. Color the points by cluster.\

f. Now do the same for every successive feature combination. ex. feature1 vs feature2, feature2 vs feature3, feature3 vs feature4, etc.\
Do this by using a 'for' loop and make sure it is reusable for other datasets.\
Tips:
- Create a figure with subplots that has n-1 axes (n being the number of features)
- loop n-1 times
    - Create a scatter-plot on the i-th axe within the loop, and plot feature i and i+1
g. Wich combination of features seperates the features in the best way?

# Customer segmentation

This exercise is based on https://www.kaggle.com/code/pablomgomez21/k-means-clustering-practice/notebook but the questions are adjusted to the workflow we use.


## Data preperation
a. Study ``Cust_segmentation.csv`` and import the data with pandas.\
b. Remove 'Adress' as it is Categorical data. Customer_Id should also be removed because it is not useful for clustering.\


### Model and hyperparameter selection

c. Fit a K-means with 3 clusters and n_init=12 on this set. Use random_state=42 to ensure the same results every time. Make sure the data is normalized before fitting.\
Mind that, even if you do this correctly, you will get an error because of NaN values. Investigate the problem and take a proper action.\


d. Display the centers of the clusters. Tip: Use ``named_steps['kmeans']`` to access the Kmeans-model object within the pipeline\
e. The problem here is that you are showing the means of the normalized dataset. Perform an ``inverse_transform()`` on the ``cluster_centers_``. ``inverse_transform()`` is availaible on the ``standardscaler`` step in  the pipeline.


In [69]:
#d


#e


f. Finally visualize the clusters by plotting 'age' and 'income'. Color the points by cluster.\